# All About REM API Statistics

In order to make estimates for the impact of an electrical upgrade
on a home, the REM API does a
[Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method)
simulation over a number of theoretical homes that closely
resemble the target home and, taken as a whole, 
probabilistically represent it. 

We do this because we generally don't know everything about each
home. We have a large database of home properties
that tells us things like when a home what built and how many square feet
it is. But it does not give us all the details that we need to accurately
predict savings. So instead we sample theoretical homes from a 
[conditional probability distribution](https://github.com/NREL/resstock/tree/develop/project_national/housing_characteristics) based on the properties
that we do know. We then predict the energy consumption for each 
sample home using a machine learning model, which yields
a distribution of outcomes that probabilistically represents the
query home's energy consumption under the baseline and upgrade scenarios.
We compute statistics across the this distribution to decide
what is likely to happen in the query home.

This means that instead of getting one answer for
how much less fuel oil the home will use or how
much money the homeowner will save each year, it generates
many answers to the question, each based on one
theoretical home.

In the return value from the API, we get statistics across
those theoretical homes including the mean, median, and 20th
and 80th percentile values for savings, emissions, and so on.

The purpose of this notebook is to illustrate how those statistics
work, describe how they are computed, and discuss how and why
they should or should not be used in particular ways.

To learn more about modeling only baseline
view the [REM Core Demo notebook](https://github.com/rewiringamerica/api_demos/tree/main/notebooks/REM%20Demo).


## Imports and Configuration

In [1]:
import requests
from pathlib import Path

In [2]:
HOST = "https://api.rewiringamerica.org"
REM_ADDRESS_URL = f"{HOST}/api/v1/rem/address"

API_KEY = None  # Put your API key here, or better yet in the file ~/.rwapi/api_key.txt

In [3]:
if API_KEY is None:
    api_key_path = Path.home() / ".rwapi" / "api_key.txt"

    if api_key_path.is_file():
        with open(api_key_path) as f:
            API_KEY = f.read().strip()

## Parameters

Address we are interested in and the upgrade we want to do.

In [4]:
address = "165 Hope St, Providence, RI 02906"
upgrade = "hvac__heat_pump_seer18_hspf10"
heating_fuel = "fuel_oil"

## Make the Request

In [5]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade=upgrade, heating_fuel=heating_fuel),
)

In [6]:
response

<Response [200]>

## Pull out the results

We are specifically interested in the total dollar savings.

In [7]:
data = response.json()

In [8]:
fuel_results = data["fuel_results"]

In [9]:
fuel_results["fuel_oil"]["baseline"]

{'energy': {'mean': {'value': 1639.9717, 'unit': 'gallon'},
  'median': {'value': 1592.8951, 'unit': 'gallon'},
  'percentile_20': {'value': 1217.5114, 'unit': 'gallon'},
  'percentile_80': {'value': 2014.3037, 'unit': 'gallon'}},
 'emissions': {'mean': {'value': 20183.0257, 'unit': 'kgCO2e'},
  'median': {'value': 19603.6575, 'unit': 'kgCO2e'},
  'percentile_20': {'value': 14983.8342, 'unit': 'kgCO2e'},
  'percentile_80': {'value': 24789.9057, 'unit': 'kgCO2e'}},
 'cost': {'mean': {'value': 6557.3876, 'unit': '$'},
  'median': {'value': 6369.1531, 'unit': '$'},
  'percentile_20': {'value': 4868.1902, 'unit': '$'},
  'percentile_80': {'value': 8054.1452, 'unit': '$'}}}

In [10]:
data["emissions_factors"]

{'electricity': {'value': 0.1241, 'unit': 'kgCO2e/kWh'},
 'natural_gas': {'value': 6.6798, 'unit': 'kgCO2e/therm'},
 'fuel_oil': {'value': 12.3069, 'unit': 'kgCO2e/gallon'},
 'propane': {'value': 7.3776, 'unit': 'kgCO2e/gallon'}}

## Let's look at fuel oil, since that's what we are replacing

It's a fairly big block of nested dictionaries, but we will go through it piece by
piece.

In [11]:
fuel_oil_results = fuel_results["fuel_oil"]

In [12]:
fuel_oil_results

{'baseline': {'energy': {'mean': {'value': 1639.9717, 'unit': 'gallon'},
   'median': {'value': 1592.8951, 'unit': 'gallon'},
   'percentile_20': {'value': 1217.5114, 'unit': 'gallon'},
   'percentile_80': {'value': 2014.3037, 'unit': 'gallon'}},
  'emissions': {'mean': {'value': 20183.0257, 'unit': 'kgCO2e'},
   'median': {'value': 19603.6575, 'unit': 'kgCO2e'},
   'percentile_20': {'value': 14983.8342, 'unit': 'kgCO2e'},
   'percentile_80': {'value': 24789.9057, 'unit': 'kgCO2e'}},
  'cost': {'mean': {'value': 6557.3876, 'unit': '$'},
   'median': {'value': 6369.1531, 'unit': '$'},
   'percentile_20': {'value': 4868.1902, 'unit': '$'},
   'percentile_80': {'value': 8054.1452, 'unit': '$'}}},
 'upgrade': {'energy': {'mean': {'value': 73.1678, 'unit': 'gallon'},
   'median': {'value': 0.0, 'unit': 'gallon'},
   'percentile_20': {'value': 0.0, 'unit': 'gallon'},
   'percentile_80': {'value': 161.4628, 'unit': 'gallon'}},
  'emissions': {'mean': {'value': 900.471, 'unit': 'kgCO2e'},
   '

The results are divided into three sections:

- `baseline` contains estimates of what was consumed in a typical year before the upgrade
- `upgrade` contains estimates of what is consumed in a typical year after the upgrade
- `delta` contains estimates of the change in consumption in a typical year due to the upgrade

In [13]:
result_keys = fuel_oil_results.keys()
result_keys

dict_keys(['baseline', 'upgrade', 'delta'])

Now we are going to pull out some mean numbers for all three.

In [14]:
def results_for_stat(results, metric, stat: str):
    """A helper function to pull out subsets of the results."""
    return {k: results[k][metric][stat] for k in result_keys}

### Mean

Let's start with the mean. For many applications, like presenting a single number to
a consumer contemplating and upgrade, this is the place we might start.

In [15]:
mean_energy = results_for_stat(fuel_oil_results, "energy", "mean")
mean_energy

{'baseline': {'value': 1639.9717, 'unit': 'gallon'},
 'upgrade': {'value': 73.1678, 'unit': 'gallon'},
 'delta': {'value': -1566.8039, 'unit': 'gallon'}}

### Mean post-upgrade consumption is not zero?

The first thing to notice is that mean consumption of fuel oil after the upgrade is not exactly zero.
It is still a small non-zero number. The reason for this is that in the sample space we constructed,
there was at least one home that used fuel oil for some purpose other than heating, like water heating,
so it continued to use it after the upgrade. In fact, while the samples are not exposed in this API,
if we look under the hood, 89 out of 200 (45%) of the theoretical samples have Fuel Oil water heating,
since heating and water heating fuels are highly correlated and fuel oil is prevalant for both in Rhode Island.

### Mean baseline, upgrade, and change

Now let's look at how the upgrade changed consumption. For the mean of the distribution, the consumption of energy after the upgrade should be the sum of the baseline and how much consumption changed.

In [16]:
round(
    mean_energy["upgrade"]["value"]
    - (mean_energy["baseline"]["value"] + mean_energy["delta"]["value"]),
    2,
)

0.0

### Median

Now let's do the same analysis, but on the median values.

In [17]:
median_energy = results_for_stat(fuel_oil_results, "energy", "median")
median_energy

{'baseline': {'value': 1592.8951, 'unit': 'gallon'},
 'upgrade': {'value': 0.0, 'unit': 'gallon'},
 'delta': {'value': -1521.6893, 'unit': 'gallon'}}

Unlike the mean, the median of the distribution uses no fuel oil after the upgrade. This is because a 
minority of homes in the sample used fuel oil for things other than heating (in this case, we know it was 45%). 

We can look at the 20th and 80th percentile and see that they are zero and non-zero 
respectively after the upgrade.

In [18]:
results_for_stat(fuel_oil_results, "energy", "percentile_20")["upgrade"]

{'value': 0.0, 'unit': 'gallon'}

In [19]:
results_for_stat(fuel_oil_results, "energy", "percentile_80")["upgrade"]

{'value': 161.4628, 'unit': 'gallon'}

## Emissions

In addition to consumption of various fuels, the model estimates in kgCO2e (which include other greenhouse such as methane) emissions before and after the upgrade. Let's look at some of those numbers for total household emissions, including those for  all fuels and all end uses in the home.

In [20]:
total_results = fuel_results["total"]

In [21]:
median_emissions = results_for_stat(total_results, "emissions", "median")

In [22]:
median_emissions

{'baseline': {'value': 21423.6426, 'unit': 'kgCO2e'},
 'upgrade': {'value': 4743.954, 'unit': 'kgCO2e'},
 'delta': {'value': -16628.4887, 'unit': 'kgCO2e'}}

### Median baseline, upgrade, and change

Now let's see if the median behaves like the mean did when we add things up.
(Spoiler alert: it does not.)

In [23]:
round(
    median_emissions["upgrade"]["value"]
    - (median_emissions["baseline"]["value"] + median_emissions["delta"]["value"]),
    2,
)

-51.2

What happened? The reason the numbers don't quite add up has to do with how we compute the medians.
`median_emissions['upgrade']` is a median taken over the total emissions of every home in the distribution
after the upgrade. `median_emissions['baseline']` is the median total emissions of every home in the distribution
before the upgrade. But because factors like insulation affect the amount of heat needed, which affects emissions
differently before and after the upgrade, homes can move around in the distribution, which can affect the median.
`median_emissions['delta']` is the median of the change in emissions, which is therefore not necessarily the difference
of the median emissions before and after the upgrade.